In [2]:
import pandas as pd
import os

In [3]:
root = "."

pathogen_code = 'mtuberculosis'

In [4]:
# Load assays info
ASSAYS = pd.read_csv(os.path.join(root, "..", "output", pathogen_code, "assays.csv"))

# Load pathogen data
ChEMBL = pd.read_csv(os.path.join(root, "..", "output", pathogen_code, f"{pathogen_code}_ChEMBL_data.csv"), low_memory=False)

In [6]:
ASSAYS

,assay_id,assay_type,assay_organism,doc_chembl_id,target_type,target_chembl_id,target_organism,activity_type,unit,activities,nan_values,cpds
0,CHEMBL4649948,F,Mycobacterium tuberculosis,CHEMBL3988442,UNCHECKED,CHEMBL612545,NaN,PERCENTEFFECT,%,93555,0,86589
1,CHEMBL4649949,F,Mycobacterium tuberculosis,CHEMBL3988442,UNCHECKED,CHEMBL612545,NaN,PERCENTEFFECT,%,101515,0,86575
2,CHEMBL4649971,F,Mycobacterium tuberculosis,CHEMBL3988442,ORGANISM,CHEMBL360,Mycobacterium tuberculosis,PERCENTEFFECT,%,68619,0,68613
3,CHEMBL4649972,F,Mycobacterium tuberculosis,CHEMBL3988442,PROTEIN COMPLEX,CHEMBL4662931,Mycobacterium tuberculosis (strain ATCC 25618 ...,PERCENTEFFECT,%,68616,0,68610
4,CHEMBL4649941,F,Mycobacterium tuberculosis,CHEMBL3988442,SINGLE PROTEIN,CHEMBL4662928,Mycobacterium tuberculosis (strain ATCC 25618 ...,PERCENTEFFECT,%,67381,0,66941
...,...,...,...,...,...,...,...,...,...,...,...,...
13582,CHEMBL5126010,B,Mycobacterium tuberculosis,CHEMBL5120950,SINGLE PROTEIN,CHEMBL5169231,Mycobacterium tuberculosis,INHIBITION,%,1,1,1
13583,CHEMBL5126009,B,Mycobacterium tuberculosis,CHEMBL5120950,SINGLE PROTEIN,CHEMBL5169231,Mycobacterium tuberculosis,INHIBITION,%,1,1,1
13584,CHEMBL5126008,B,Mycobacterium tuberculosis,CHEMBL5120950,SINGLE PROTEIN,CHEMBL5169231,Mycobacterium tuberculosis,INHIBITION,%,1,1,1
13585,CHEMBL5126007,B,Mycobacterium tuberculosis,CHEMBL5120950,SINGLE PROTEIN,CHEMBL5169231,Mycobacterium tuberculosis,INHIBITION,%,1,1,1


In [7]:
for assay_id, activity_type, unit in ASSAYS[['assay_id', 'activity_type', 'unit']].values:
    print(assay_id, activity_type, unit)
    break

CHEMBL4649948 PERCENTEFFECT %


In [10]:
ChEMBL[(ChEMBL['assay_chembl_id'] == assay_id) & (ChEMBL['activity_type'] == activity_type) & (ChEMBL['unit'] == unit)]

,activity_id,assay_id,assay_chembl_id,assay_type,assay_confidence_score,assay_organism,doc_chembl_id,tid,target_type,target_organism,...,canonical_smiles,MW,pchembl,activity_comment,standard_text,value,unit,activity_type,relation,pchembl_calculated
103002,20807772,1998109,CHEMBL4649948,F,0,Mycobacterium tuberculosis,CHEMBL3988442,22226,UNCHECKED,NaN,...,Cn1cc(CCN2CCOC(CNc3cccnn3)C2)cn1,302.382,NaN,0,0,-33.180,%,PERCENTEFFECT,=,NaN
103003,20807773,1998109,CHEMBL4649948,F,0,Mycobacterium tuberculosis,CHEMBL3988442,22226,UNCHECKED,NaN,...,CC(=O)N(C)C(c1cccc(F)c1)c1ccccn1,258.296,NaN,0,0,-3.704,%,PERCENTEFFECT,=,NaN
103004,20807774,1998109,CHEMBL4649948,F,0,Mycobacterium tuberculosis,CHEMBL3988442,22226,UNCHECKED,NaN,...,CCc1noc(CN2CCN(c3nccs3)CC2)n1,279.369,NaN,0,0,-28.810,%,PERCENTEFFECT,=,NaN
103005,20807775,1998109,CHEMBL4649948,F,0,Mycobacterium tuberculosis,CHEMBL3988442,22226,UNCHECKED,NaN,...,Cc1nc(C2CCCN2C(=O)CCCn2cncn2)no1,290.327,NaN,0,0,-15.040,%,PERCENTEFFECT,=,NaN
103006,20807776,1998109,CHEMBL4649948,F,0,Mycobacterium tuberculosis,CHEMBL3988442,22226,UNCHECKED,NaN,...,OCCN1CCN(Cc2coc(-c3ccccc3)n2)CC1,287.363,NaN,0,0,-9.754,%,PERCENTEFFECT,=,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695743,23048890,1998109,CHEMBL4649948,F,0,Mycobacterium tuberculosis,CHEMBL3988442,22226,UNCHECKED,NaN,...,Nc1ccn(Cc2ccccc2)n1,173.219,NaN,0,0,19.050,%,PERCENTEFFECT,=,NaN
695744,23048891,1998109,CHEMBL4649948,F,0,Mycobacterium tuberculosis,CHEMBL3988442,22226,UNCHECKED,NaN,...,Cn1cc(C(=O)NCc2cccc(F)c2)ccc1=O,260.268,NaN,0,0,-7.690,%,PERCENTEFFECT,=,NaN
695745,23048892,1998109,CHEMBL4649948,F,0,Mycobacterium tuberculosis,CHEMBL3988442,22226,UNCHECKED,NaN,...,CN(C)C(=O)NCCNc1nc(-c2ccccc2)ns1,291.380,NaN,0,0,-12.060,%,PERCENTEFFECT,=,NaN
695746,23048893,1998109,CHEMBL4649948,F,0,Mycobacterium tuberculosis,CHEMBL3988442,22226,UNCHECKED,NaN,...,CNCc1nc(C(F)(F)F)cs1,196.197,NaN,0,0,4.224,%,PERCENTEFFECT,=,NaN
